In [1]:
%matplotlib inline

In [2]:
import pandas as pd
import os
import re
from Bio import SeqIO
from Bio import SeqUtils
import pysam
from Bio.SeqRecord import SeqRecord
from pybedtools import BedTool
import numpy as np
import pybedtools
import time
import matplotlib.pyplot as plt
import sys
import subprocess
import shutil
from Bio.Seq import Seq
import pysam
from Bio import SearchIO
import json
import glob
import scipy.stats as stats
import statsmodels as sms
import statsmodels.sandbox.stats.multicomp
import distance
import seaborn as sns
from pybedtools import BedTool
import matplotlib
from sklearn.externals.joblib import Parallel, delayed
import itertools as it
import tempfile
from scipy.signal import argrelextrema
import scipy
from IPython.display import Image
from PIL import Image
from collections import OrderedDict

/home/benjamin/anaconda3/lib/python3.5/site-packages/Bio/SearchIO/__init__.py:211: BiopythonExperimentalWarning: Bio.SearchIO is an experimental submodule which may undergo significant changes prior to its future official release.
  BiopythonExperimentalWarning)


In [3]:
#now do some folder as input
BASEFOLDER = '/home/benjamin/genome_assembly/PST79/FALCON/p_assemblies/v9_1/Pst_104E_v12/Warrior_comp_runs/mummer_comp'
GENOME_FOLDER = os.path.join(BASEFOLDER, 'genomes')
OUTFOLDER = os.path.join(BASEFOLDER, 'output')
if not os.path.exists(OUTFOLDER):
    os.makedirs(OUTFOLDER)
DK0911_p = 'DK_0911_v04_p_ctg.fa'
DK0911_h = 'DK_0911_v04_h_ctg.fa'
DK0911_ph = 'DK_0911_v04_ph_ctg.fa'
PST_104E_p = 'Pst_104E_v13_p_ctg.fa'
PST_104E_h = 'Pst_104E_v13_h_ctg.fa'
PST_104E_ph = 'Pst_104E_v13_ph_ctg.fa'

In [4]:
counter = 0
contig_series = pd.Series()
length_series = pd.Series()
for seq in SeqIO.parse(fa_fn, 'fasta'):
    length_series.set_value(counter, len(seq.seq))
    contig_series.set_value(counter, seq.id)
    counter = counter + 1
tmp_df = pd.concat([contig_series, length_series], axis=1)
tmp_df['start'] = 0
tmp_df['stop'] = tmp_df[1] -1
tmp_df.sort_values(0, inplace=True)
tmp_df.loc[:, [0, 'start', 'stop']].to_csv(fa_fn.replace('.fa', '.contig.bed'), sep='\t', header=None, index=None)

NameError: name 'fa_fn' is not defined

In [5]:
def contig_bed(fa_fn):
    from Bio import SeqIO
    counter = 0
    contig_series = pd.Series()
    length_series = pd.Series()
    for seq in SeqIO.parse(fa_fn, 'fasta'):
        length_series.set_value(counter, len(seq.seq))
        contig_series.set_value(counter, seq.id)
        counter = counter + 1
    tmp_df = pd.concat([contig_series, length_series], axis=1)
    tmp_df['start'] = 0
    tmp_df['stop'] = tmp_df[1] -1
    tmp_df.sort_values(0, inplace=True)
    tmp_df.loc[:, [0, 'start', 'stop']].to_csv(fa_fn.replace('.fa', '.contig.bed'), sep='\t', header=None, index=None)
    return fa_fn.replace('.fa', '.contig.bed')

In [6]:
def run_command(command, v=False):
    #print('\nRunnning now!\n')
    if v == True:
        print(command)
    try:
        output = subprocess.check_output(command, shell=True, stderr=subprocess.STDOUT)
    except subprocess.CalledProcessError as ex:
        o = ex.output
        returncode = ex.returncode
        if returncode == 25:
            next
        else:
            raise
    #print("Done with it!")
    #print('\nDone\nWith ouput:\n%s' % output)
    #print(output)

In [7]:
def run_nucmer(ref, query,maxmatch=True, l=100, c=500):
    '''Run nucmer with three options.'''
    #print(os.path.basename(ref)[:-3])
    delta = os.path.join(OUTFOLDER, '%s.%s.l%s.c%s.delta' % (os.path.basename(ref)[:-3], os.path.basename(query)[:-3], l,c))
    command='/home/benjamin/anaconda3/mummer-4.0.0/bin/nucmer -l %s -c %s -t 10 %s %s --prefix=%s' % (l, c, ref, query, delta.replace('.delta', ''))
    if maxmatch == True:
        delta = os.path.join(OUTFOLDER, '%s.%s.l%s.c%s.mm.delta' % (os.path.basename(ref)[:-3], os.path.basename(query)[:-3], l,c))
        command='nucmer --maxmatch -l %s -c %s %s %s --prefix=%s' % (l, c, ref, query, delta.replace('.delta', ''))
    if not os.path.exists(delta):
        run_command(command)
    return delta

In [8]:
def run_delta_filter(delta, v=False, **kwargs):
    filter_string = ''
    if kwargs is not None:
        for key,value in kwargs.items():
            if key in ['1', 'g', 'h', 'm', 'q', 'r']:
                filter_string = '%s %s' % (filter_string, value)
            elif key in ['i', 'l', 'u', 'o']:
                filter_string = '%s -%s %s' % (filter_string, key, value)
            if v == True:
                print('Used following delta filter option: %s %s' % (key,value))
                print(filter_string)
            
            
    delta_filter = delta.replace('.delta', '.deltafilter_%s' %(filter_string.replace(' ', '').replace('-','')))
    command = '/home/benjamin/anaconda3/mummer-4.0.0/bin/delta-filter %s %s > %s' % (filter_string, delta, delta_filter)
    run_command(command)
    return delta_filter

In [9]:
def run_show_coords(delta_filter,v=False, **kwargs):
    filter_string = ''
    if kwargs is not None:
        for key,value in kwargs.items():
            if v==True:
                print('Used following show-coords option: %s %s' % (key,value))
            filter_string = '%s %s' % (filter_string, value)
    #print(delta_filter.index('.delta'))
    showcoords = '%s_%s.coords' % (delta_filter, filter_string.replace(' ', '').replace('-',''))
    command = '/home/benjamin/anaconda3/mummer-4.0.0/bin/show-coords -T %s %s > %s' % (filter_string, delta_filter, showcoords)
    run_command(command)
    return showcoords

In [10]:
def run_mummerplot(delta_filter,v=False, **kwargs):
    filter_string = ''
    if kwargs is not None:
        for key,value in kwargs.items():
            if v == True:
                print('Used following mummerplot option: %s %s' % (key,value))
            filter_string = '%s %s' % (filter_string, value)
    mummerplot = os.path.basename(delta_filter)
    if 'r' in kwargs.keys():
            mummerplot = mummerplot + kwargs['r'].replace('-r ','_')
    previous_dir = os.getcwd()
    os.chdir(OUTFOLDER)
    command = '/home/benjamin/anaconda3/mummer-4.0.0/bin/mummerplot %s -p %s --png %s' % (filter_string, mummerplot, delta_filter)
    run_command(command)
    os.chdir(previous_dir)
    return '%s.png' % (os.path.join(OUTFOLDER, mummerplot))

In [11]:
def show_cov_plot(mummerplot, show=False):
    if not os.path.exists(mummerplot):
        return None
    image = Image.open(mummerplot)
    image = image.crop((10,500,800,800))
    print('\n##########\n')
    print('This is the coverage plot %s!' % os.path.basename(mummerplot))
    print('\n##########\n')
    if show==True:
        image
    return image

In [12]:
def show_align_plot(mummerplot, show=False):
    if not os.path.exists(mummerplot):
        return None
    image = Image.open(mummerplot)
    print('\n##########\n')
    print('This is the alignment plot %s!' % os.path.basename(mummerplot))
    print('\n##########\n')
    if show==True:
        image
    return image

In [13]:
def nrbed_reverse(input_bed_fn, genome_bed_fn):
    genome_bed = BedTool(genome_bed_fn)
    input_bed = BedTool(input_bed_fn)
    out_fn = input_bed_fn.replace('.nr.bed', '.nr_reverse.bed')
    genome_bed.subtract(input_bed).saveas(out_fn)
    return out_fn

In [14]:
def coords_to_nrbed(coords_fn, merged_d = 0):
    with open(coords_fn, 'r') as fh:
        counter = 0
        for line in fh:
            #if line.startswith('['):
                #counter = 1
            #if counter == 1:
            n_columns = len(line.split('\t'))
    column_names = ['r_start','r_stop', 'q_start', 'q_stop', 'aln_len_r', 'aln_len_q', 'pc_id', 'len_r', 'len_q', 'ref_name', 'query_name']

    if len(column_names) != n_columns:
        column_names.remove('len_r' )
        column_names.remove('len_q')

    df = pd.read_csv(coords_fn, sep='\t', skiprows=4, names=column_names )
    tmp_bed_fn = coords_fn.replace('.coords', '.tmpbed')
    df['r_bed_start'] = df.r_start - 1
    df['r_bed_stop'] = df.r_stop - 1
    df.loc[:, ['ref_name', 'r_bed_start', 'r_bed_stop']].to_csv(tmp_bed_fn, header=None, index=None, sep='\t')
    BedTool(tmp_bed_fn).sort().merge(d=merged_d).saveas(tmp_bed_fn.replace('.tmpbed', '.nr.bed'))
    return tmp_bed_fn.replace('.tmpbed', '.nr.bed')

In [15]:
def count_bases_in_bed(fn):
    """Function that sums the interval size of bedfile."""
    header = ['contig', 'start', 'stop']
    df = pd.read_csv(fn, sep='\t', header=None, names=header)
    df['interval'] = df.stop - df.start
    return df.interval.sum()

In [16]:
#make the contig bed files for substraction later on 
contig_bed_fn_dict = {}
for fa_fn in [os.path.join(GENOME_FOLDER, x) for x in os.listdir(GENOME_FOLDER)]:
    if fa_fn.endswith('.fa'):
        contig_bed_fn_dict[fa_fn.split('/')[-1][:-3]] = contig_bed(fa_fn)

/home/benjamin/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:7: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/home/benjamin/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:8: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/home/benjamin/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:7: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/home/benjamin/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:8: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


In [17]:
delta_DK0911_self_nmm = run_nucmer(os.path.join(GENOME_FOLDER, DK0911_p), os.path.join(GENOME_FOLDER, DK0911_h), maxmatch=False)
delta_DK0911_Pst_104Ep_nmm = run_nucmer(os.path.join(GENOME_FOLDER, DK0911_p), os.path.join(GENOME_FOLDER, PST_104E_p), maxmatch=False)
delta_DK0911_Pst_104Eph_nmm = run_nucmer(os.path.join(GENOME_FOLDER, DK0911_p), os.path.join(GENOME_FOLDER, PST_104E_ph), maxmatch=False)
delta_Pst_104E_DK0911ph_nmm = run_nucmer(os.path.join(GENOME_FOLDER, PST_104E_p), os.path.join(GENOME_FOLDER, DK0911_ph), maxmatch=False)
delta_Pst_104E_self_nmm = run_nucmer(os.path.join(GENOME_FOLDER, PST_104E_p), os.path.join(GENOME_FOLDER, PST_104E_h), maxmatch=False)
delta_Pst_104E_DK0911p_nmm = run_nucmer(os.path.join(GENOME_FOLDER, PST_104E_p), os.path.join(GENOME_FOLDER, DK0911_p), maxmatch=False)

In [24]:
delta_Pst_104Eph_DK0911ph_mm = run_nucmer(os.path.join(GENOME_FOLDER, PST_104E_ph), os.path.join(GENOME_FOLDER, DK0911_ph))
delta_DK0911ph_Pst_104Eph_mm = run_nucmer(os.path.join(GENOME_FOLDER, DK0911_ph), os.path.join(GENOME_FOLDER, PST_104E_ph))

delta_Pst_104Eph_Pst_104Eph_mm = run_nucmer(os.path.join(GENOME_FOLDER, PST_104E_ph), os.path.join(GENOME_FOLDER, PST_104E_ph))
delta_DK0911ph_DK0911ph_mm = run_nucmer(os.path.join(GENOME_FOLDER, DK0911_ph), os.path.join(GENOME_FOLDER, DK0911_ph))

In [ ]:
#run a couple of delta mapping for mummer
delta_DK0911_self_mm = run_nucmer(os.path.join(GENOME_FOLDER, DK0911_p), os.path.join(GENOME_FOLDER, DK0911_h))
delta_DK0911_Pst_104Ep_mm = run_nucmer(os.path.join(GENOME_FOLDER, DK0911_p), os.path.join(GENOME_FOLDER, PST_104E_p))
delta_DK0911_Pst_104Eph_mm = run_nucmer(os.path.join(GENOME_FOLDER, DK0911_p), os.path.join(GENOME_FOLDER, PST_104E_ph))

In [ ]:
delta_Pst_104E_DK0911p_mm = run_nucmer(os.path.join(GENOME_FOLDER, PST_104E_p), os.path.join(GENOME_FOLDER, DK0911_p))
delta_Pst_104E_DK0911ph_mm = run_nucmer(os.path.join(GENOME_FOLDER, PST_104E_p), os.path.join(GENOME_FOLDER, DK0911_ph))
delta_Pst_104E_self_mm = run_nucmer(os.path.join(GENOME_FOLDER, PST_104E_p), os.path.join(GENOME_FOLDER, PST_104E_h))

In [ ]:
show_cov_plot(run_mummerplot(run_delta_filter(delta_DK0911_self_mm, **{'m':'-m', 'i': 95}, v=True), **{'c': '-c', 'r': '-r pcontig_000'}))

In [ ]:
show_cov_plot(run_mummerplot(run_delta_filter(delta_DK0911_self_mm, **{'m':'-m', 'i': 98}, v=True), **{'c': '-c', 'r': '-r pcontig_000'}))

In [ ]:
show_cov_plot(run_mummerplot(run_delta_filter(delta_DK0911_self_nmm, **{'m':'-m'}), **{'c': '-c', 'r': '-r pcontig_000'}))

In [ ]:
show_cov_plot(run_mummerplot(run_delta_filter(delta_DK0911_Pst_104Ep_mm, **{'m':'-m'}), **{'c': '-c', 'r': '-r pcontig_000'}))

In [ ]:
show_cov_plot(run_mummerplot(run_delta_filter(delta_DK0911_Pst_104Eph_mm, **{'m':'-m'}), **{'c': '-c', 'r': '-r pcontig_000'}))

In [ ]:
show_cov_plot(run_mummerplot(run_delta_filter(delta_DK0911_Pst_104Eph_mm, **{'m':'-m', 'i': '99'}), **{'c': '-c', 'r': '-r pcontig_002'}))

In [ ]:
show_cov_plot(run_mummerplot(run_delta_filter(delta_DK0911_Pst_104Eph_mm, **{'m':'-m', 'i': '80'}), **{'c': '-c', 'r': '-r pcontig_020'}))

In [ ]:
show_cov_plot(run_mummerplot(run_delta_filter(delta_Pst_104E_DK0911ph_mm, **{'m':'-m', 'i': '98'}), **{'c': '-c', 'r': '-r pcontig_010'}))

In [ ]:
Pst_104_p_contig_dict = {}
for seq in SeqIO.parse(os.path.join(GENOME_FOLDER, PST_104E_p), 'fasta'):
    Pst_104_p_contig_dict[seq.id] = len(seq.seq)
Pst_104_p_contig_dict = OrderedDict(sorted(Pst_104_p_contig_dict.items()))
for seq, length in Pst_104_p_contig_dict.items():
    print('contig %s of DK0911p with %i length.' %(seq, length))
    image = show_align_plot(run_mummerplot(run_delta_filter(delta_Pst_104E_DK0911p_mm, **{'m':'-m', 'i': '98', 'u': '0'}), **{ 'r': '-r %s' % seq }))
    display(image)

In [ ]:
DK0911_p_contig_dict = {}
for seq in SeqIO.parse(os.path.join(GENOME_FOLDER, DK0911_p), 'fasta'):
    DK0911_p_contig_dict[seq.id] = len(seq.seq)
DK0911_p_contig_dict = OrderedDict(sorted(DK0911_p_contig_dict.items()))
for seq, length in DK0911_p_contig_dict.items():
    print('contig %s of Pst_104_p with %i length.' %(seq, length))
    image = show_align_plot(run_mummerplot(run_delta_filter(delta_DK0911_Pst_104Ep_mm, **{'m':'-m', 'i': '98', 'u': '0'}), **{ 'r': '-r %s' % seq }))
    display(image)

Now generate some bed files that contain the overlaps and the non-overlaps for either mapping.
The cut-offs are 70, 80, 85, 90, 95, 99, 100. Also print the number of non-overlap bases.

In [ ]:
nucmer_files= [delta_Pst_104Eph_DK0911ph_mm, delta_DK0911ph_Pst_104Eph_mm ,delta_Pst_104Eph_Pst_104Eph_mm, delta_DK0911ph_DK0911ph_mm]
identidies = [70, 80, 85, 90, 95, 96, 97, 98, 99, 100]
for nucmer_file in nucmer_files:
    genome = nucmer_file.split('/')[-1].split('.')[0]
    genome_bed_fn = contig_bed_fn_dict[genome]
    for id in identidies:
        nr_bed_fn = coords_to_nrbed(run_show_coords(run_delta_filter(nucmer_file, **{'m':'-m', 'i': id, 'u': '0', 'g': '-g'})))
        nr_reverse_bed_fn = nrbed_reverse(nr_bed_fn, genome_bed_fn)
        non_covered_bases = count_bases_in_bed(nr_reverse_bed_fn)
        total_bases = count_bases_in_bed(genome_bed_fn)
        print('The genome %s with the nucmer alignment file %s has %i bases missing (%.2f percent) with and mimimum identify of %i' %\
             (genome, nucmer_file.split('/')[-1], non_covered_bases, (non_covered_bases/total_bases*100), id))

### Here I am looking at the dnadiff output of DK0911 as ref and Pst104E as query run on the command line

I guess the question here is why is the aligned sequence in the out.report so high while the non-covered region out of the coords file is also fairly high. 

In [ ]:
merged_d = 0
coords_fn = '/home/benjamin/genome_assembly/PST79/FALCON/p_assemblies/v9_1/Pst_104E_v12/Warrior_comp_runs/mummer_comp/dnadiff/Pst104E_on_DK0911/out.mcoords'
with open(coords_fn, 'r') as fh:
    counter = 0
    for line in fh:
        #if line.startswith('['):
            #counter = 1
        #if counter == 1:
        n_columns = len(line.split('\t'))
column_names = ['r_start','r_stop', 'q_start', 'q_stop', 'aln_len_r', 'aln_len_q', 'pc_id', 'len_r', 'len_q', 'pc_aln_r', 'pc_aln_q', 'ref_name', 'query_name']

if len(column_names) != n_columns:
    column_names.remove('len_r' )
    column_names.remove('len_q')

df = pd.read_csv(coords_fn, sep='\t', names=column_names )
tmp_bed_fn = coords_fn + '.tmpbed'
df['r_bed_start'] = df.r_start.astype(int) - 1
df['r_bed_stop'] = df.r_stop.astype(int) - 1
df.loc[:, ['ref_name', 'r_bed_start', 'r_bed_stop']].to_csv(tmp_bed_fn, header=None, index=None, sep='\t')
BedTool(tmp_bed_fn).sort().merge(d=merged_d).saveas(tmp_bed_fn.replace('.tmpbed', '.nr.bed'))


In [ ]:
count_bases_in_bed(tmp_bed_fn.replace('.tmpbed', '.nr.bed'))

In [ ]:
nrbed_fn = nrbed_reverse(tmp_bed_fn.replace('.tmpbed', '.nr.bed'), contig_bed_fn_dict['DK_0911_v04_ph_ctg'])

In [ ]:
count_bases_in_bed(nrbed_fn)

In [ ]:
contig_bed_fn_dict['DK_0911_v04_ph_ctg']

### Runs some tests here for alignments and delta filter 1

* look at regular delta alignments  maxmatch=False, l=20, c=65

In [ ]:
delta_Pst_104Eph_DK0911ph_mm = run_nucmer(os.path.join(GENOME_FOLDER, PST_104E_ph), os.path.join(GENOME_FOLDER, DK0911_ph))
delta_DK0911ph_Pst_104Eph_mm = run_nucmer(os.path.join(GENOME_FOLDER, DK0911_ph), os.path.join(GENOME_FOLDER, PST_104E_ph))

In [22]:
nucmer_files= [delta_Pst_104Eph_DK0911ph_mm, delta_DK0911ph_Pst_104Eph_mm , delta_DK0911ph_DK0911ph_mm, delta_Pst_104Eph_Pst_104Eph_mm]
identidies = [70, 80, 85, 90, 95, 96, 97, 98, 99, 100]
for nucmer_file in nucmer_files:
    genome = nucmer_file.split('/')[-1].split('.')[0]
    genome_bed_fn = contig_bed_fn_dict[genome]
    for id in identidies:
        nr_bed_fn = coords_to_nrbed(run_show_coords(run_delta_filter(nucmer_file, **{'m':'-m', 'i': id, 'u': '0'})))
        nr_reverse_bed_fn = nrbed_reverse(nr_bed_fn, genome_bed_fn)
        non_covered_bases = count_bases_in_bed(nr_reverse_bed_fn)
        total_bases = count_bases_in_bed(genome_bed_fn)
        print('The genome %s with the nucmer alignment file %s has %i bases missing (%.2f percent) with and mimimum identify of %i' %\
             (genome, nucmer_file.split('/')[-1], non_covered_bases, (non_covered_bases/total_bases*100), id))

The genome Pst_104E_v13_ph_ctg with the nucmer alignment file Pst_104E_v13_ph_ctg.DK_0911_v04_ph_ctg.l100.c500.mm.delta has 14951425 bases missing (9.53 percent) with and mimimum identify of 70
The genome Pst_104E_v13_ph_ctg with the nucmer alignment file Pst_104E_v13_ph_ctg.DK_0911_v04_ph_ctg.l100.c500.mm.delta has 14951425 bases missing (9.53 percent) with and mimimum identify of 80
The genome Pst_104E_v13_ph_ctg with the nucmer alignment file Pst_104E_v13_ph_ctg.DK_0911_v04_ph_ctg.l100.c500.mm.delta has 14951425 bases missing (9.53 percent) with and mimimum identify of 85
The genome Pst_104E_v13_ph_ctg with the nucmer alignment file Pst_104E_v13_ph_ctg.DK_0911_v04_ph_ctg.l100.c500.mm.delta has 14966920 bases missing (9.54 percent) with and mimimum identify of 90
The genome Pst_104E_v13_ph_ctg with the nucmer alignment file Pst_104E_v13_ph_ctg.DK_0911_v04_ph_ctg.l100.c500.mm.delta has 15406185 bases missing (9.82 percent) with and mimimum identify of 95
The genome Pst_104E_v13_ph_ctg

In [ ]:
nucmer_files= [run_nucmer(os.path.join(GENOME_FOLDER, PST_104E_ph),os.path.join(GENOME_FOLDER, DK0911_ph), maxmatch=True, l=20, c=65),\
               run_nucmer(os.path.join(GENOME_FOLDER, DK0911_ph), os.path.join(GENOME_FOLDER, PST_104E_ph), maxmatch=True, l=20, c=65)]
identidies = [70, 80, 85, 90, 95, 96, 97, 98, 99, 100]
for nucmer_file in nucmer_files:
    genome = nucmer_file.split('/')[-1].split('.')[0]
    genome_bed_fn = contig_bed_fn_dict[genome]
    for id in identidies:
        nr_bed_fn = coords_to_nrbed(run_show_coords(run_delta_filter(nucmer_file, **{'m':'-m', 'i': id, 'u': '0'})))
        nr_reverse_bed_fn = nrbed_reverse(nr_bed_fn, genome_bed_fn)
        non_covered_bases = count_bases_in_bed(nr_reverse_bed_fn)
        total_bases = count_bases_in_bed(genome_bed_fn)
        print('The genome %s with the nucmer alignment file %s has %i bases missing (%.2f percent) with and mimimum identify of %i' %\
             (genome, nucmer_file.split('/')[-1], non_covered_bases, (non_covered_bases/total_bases*100), id))

In [ ]:
delta = '/home/benjamin/genome_assembly/PST79/FALCON/p_assemblies/v9_1/Pst_104E_v12/Warrior_comp_runs/mummer_comp/dnadiff/Pst104E_on_DK0911/out.delta'
nucmer_files = [delta]
identidies = [70, 80, 85, 90, 95, 96, 97, 98, 99, 100]
for nucmer_file in nucmer_files:
    genome = 'DK_0911_v04_ph_ctg'
    genome_bed_fn = contig_bed_fn_dict[genome]
    for id in identidies:
        nr_bed_fn = coords_to_nrbed(run_show_coords(run_delta_filter(nucmer_file, **{'m':'-m', 'i': id, 'u': '0'})))
        nr_reverse_bed_fn = nrbed_reverse(nr_bed_fn, genome_bed_fn)
        non_covered_bases = count_bases_in_bed(nr_reverse_bed_fn)
        total_bases = count_bases_in_bed(genome_bed_fn)
        print('The genome %s with the nucmer alignment file %s has %i bases missing (%.2f percent) with and mimimum identify of %i' %\
             (genome, nucmer_file.split('/')[-1], non_covered_bases, (non_covered_bases/total_bases*100), id))

In [ ]:
print('Hello')

In [ ]:
delta = '/home/benjamin/genome_assembly/PST79/FALCON/p_assemblies/v9_1/Pst_104E_v12/Warrior_comp_runs/mummer_comp/dnadiff/Pst104E_on_DK0911/out.delta'
nucmer_files = [delta]
identidies = [70, 80, 85, 90, 95, 96, 97, 98, 99, 100]
for nucmer_file in nucmer_files:
    genome = 'DK_0911_v04_ph_ctg'
    genome_bed_fn = contig_bed_fn_dict[genome]
    for id in identidies:
        nr_bed_fn = coords_to_nrbed(run_show_coords(run_delta_filter(nucmer_file, **{'m':'-m', 'i': id, 'u': '0'})))
        nr_reverse_bed_fn = nrbed_reverse(nr_bed_fn, genome_bed_fn)
        non_covered_bases = count_bases_in_bed(nr_reverse_bed_fn)
        total_bases = count_bases_in_bed(genome_bed_fn)
        print('The genome %s with the nucmer alignment file %s has %i bases missing (%.2f percent) with and mimimum identify of %i' %\
             (genome, nucmer_file.split('/')[-1], non_covered_bases, (non_covered_bases/total_bases*100), id))

In [ ]:
!ls {OUTFOLDER}

In [ ]:
print(genome, nucmer_file.split('/')[-1], non_covered_bases, (non_covered_bases/total_bases*100), id)

In [ ]:
total_bases

In [ ]:
genome_bed_fn 

In [ ]:
contig_bed_fn_dict

In [ ]:
coords_fn = '/home/benjamin/genome_assembly/PST79/FALCON/p_assemblies/v9_1/Pst_104E_v12/Warrior_comp_runs/mummer_comp/output/DK_0911_v04_p_ctg.DK_0911_v04_h_ctg.l100.c500_rl.coords'

In [ ]:
coords_to_nrbed(run_show_coords(run_delta_filter(delta_Pst_104Eph_DK0911ph_mm, **{'m':'-m', 'i': '98', 'u': '0'})))

In [ ]:
!head /home/benjamin/genome_assembly/PST79/FALCON/p_assemblies/v9_1/Pst_104E_v12/Warrior_comp_runs/mummer_comp/output/Pst_104E_v13_ph_ctg.DK_0911_v04_ph_ctg.l100.c500.mm.deltafilter_u0mi98_.nr.bed

In [ ]:
coords_to_nrbed(run_show_coords(run_delta_filter(delta_Pst_104Eph_DK0911ph_mm, **{'m':'-m', 'i': '98', 'u': '0'})))

In [ ]:
count_bases_in_bed(coords_to_nrbed(run_show_coords(run_delta_filter(delta_Pst_104Eph_DK0911ph_mm, **{'m':'-m', 'i': '98', 'u': '0'}))))

In [ ]:
count_bases_in_bed(coords_to_nrbed(run_show_coords(run_delta_filter(delta_Pst_104Eph_DK0911ph_mm, **{'m':'-m', 'i': '99', 'u': '0'})) , merged_d=100))

In [ ]:
count_bases_in_bed(coords_to_nrbed(run_show_coords(run_delta_filter(delta_Pst_104Eph_DK0911ph_mm, **{'m':'-m', 'i': '99', 'u': '0'})) , merged_d=0))

In [ ]:
!ls /home/benjamin/genome_assembly/PST79/FALCON/p_assemblies/v9_1/Pst_104E_v12/Warrior_comp_runs/mummer_comp/genomes/

In [ ]:
coords_to_nrbed(run_show_coords(run_delta_filter(delta_Pst_104Eph_DK0911ph_mm, **{'m':'-m', 'i': '98', 'u': '0'})))

In [ ]:
tmp_bed_object = BedTool(coords_to_nrbed(run_show_coords(run_delta_filter(delta_Pst_104Eph_DK0911ph_mm, **{'m':'-m', 'i': '98', 'u': '0'}))))

In [ ]:
tmp_bed_object.head()

In [ ]:
tmp_bed_object.merge(d=100).head()

In [ ]:
!head {coords_fn}

In [ ]:
with open(coords_fn, 'r') as fh:
    counter = 0
    for line in fh:
        #if line.startswith('['):
            #counter = 1
        #if counter == 1:
        n_columns = len(line.split('\t'))
column_names = ['r_start','r_stop', 'q_start', 'q_stop', 'aln_len_r', 'aln_len_q', 'pc_id', 'len_r', 'len_q', 'ref_name', 'query_name']

if len(columns) != n_columns:
    column_names.remove('len_r' )
    column_names.remove('len_q')

df = pd.read_csv(coords_fn, sep='\t', skiprows=4, names=column_names )
tmp_bed_fn = coords_fn.replace('.coords', '.tmpbed')
df['r_bed_start'] = df.r_start - 1
df['r_bed_stop'] = df.r_stop - 1
df.loc[:, ['ref_name', 'r_bed_start', 'r_bed_stop']].to_csv(tmp_bed_fn, header=None, index=None, sep='\t')
BedTool(tmp_bed_fn).sort().merge().saveas(tmp_bed_fn.replace('.tmpbed', '.nr.bed'))
return tmp_bed_fn.replace('.tmpbed', '.nr.bed')

In [ ]:
count_bases_in_bed

In [ ]:
df

In [ ]:
image = show_align_plot(run_mummerplot(run_delta_filter(delta_Pst_104E_DK0911p_mm, **{'m':'-m', 'i': '-i 90', 'u': '-u 0'}), **{ 'r': '-r pcontig_193'}))

In [ ]:
fig, ax = plt.subplots(2)
ax[1] = image

In [ ]:
show_align_plot(run_mummerplot(run_delta_filter(delta_Pst_104E_DK0911p_mm, **{'m':'-m', 'i': '-i 90', 'u': '-u 0'}), **{ 'r': '-r pcontig_193'}))

In [ ]:
show_cov_plot(run_mummerplot(run_delta_filter(delta_Pst_104E_DK0911ph_mm, **{'m':'-m', 'i': '-i 98', 'u': '-u 0'}), **{'c': '-c', 'r': '-r pcontig_003'}))

In [ ]:
show_align_plot(run_mummerplot(run_delta_filter(delta_Pst_104E_DK0911p_mm, **{'m':'-m', 'i': '-i 90', 'u': '-u 90'}), **{'r': '-r pcontig_000'}))

In [ ]:
delta_filter = {'m':'-m'}
showcoords_fiter = {}
mummerplot_filter = {'c': '-c', 'r': '-r pcontig_000'}

In [ ]:
from PIL import Image

# we get the image

image = Image.open(run_mummerplot(run_delta_filter(delta_DK0911_self_mm, **{'m':'-m'}), **{'c': '-c', 'r': '-r pcontig_000'}))

In [ ]:
image.size

In [ ]:
image.crop((10,500,800,800))

In [ ]:
display(Image(run_mummerplot(run_delta_filter(delta_DK0911_self_mm, **{'m':'-m'}), **{'c': '-c', 'r': '-r pcontig_000'})))
display(Image(run_mummerplot(run_delta_filter(delta_DK0911_self_mm), **{'c': '-c', 'r': '-r pcontig_000'})))
display(Image(run_mummerplot(run_delta_filter(delta_DK0911_self_nmm), **{'c': '-c', 'r': '-r pcontig_000'})))

In [ ]:
os.

In [ ]:
display(Image(run_mummerplot(run_delta_filter(delta_DK0911_self_mm), **{'c': '-c', 'r': '-r pcontig_000'})))

In [ ]:
np_image = [img2array(im) for im in images]

In [ ]:
np_image 

In [ ]:
for img in np_image:
    plt.figure()
    plt.imshow(img)

In [ ]:
display(Image(run_mummerplot(run_delta_filter(delta_DK0911_self_mm, **{'m':'-m'}), **{'c': '-c', 'r': '-r pcontig_000'}), width=1000, height=300))

In [ ]:
delta_filters = {'m':'-m'}
delta_filters = {}
show_coords_filter = {'l': '-l', 'r': '-r'}
mummerplot_filter = {'c': '-c', 'r': '-r pcontig_000'}

In [ ]:
Image(run_mummerplot(run_delta_filter(delta_DK0911_self_mm, **{'m':'-m'}), **{'c': '-c', 'r': '-r pcontig_000'}), width=1000, height=300)

In [ ]:
Image(run_mummerplot(run_delta_filter(delta_DK0911_self_mm), **{'c': '-c', 'r': '-r pcontig_000'}))

In [ ]:
showcoords = run_show_coords(delta_filter, **show_coords_filter)

In [ ]:
mummer_plot = run_mummerplot(delta_filter, **mummerplot_filter)

In [ ]:
previous_dir = os.getcwd()
os.chdir(OUTFOLDER)
Image(mummer_plot)
#os.chdir(previous_dir)